# 3. Medisinsk Bildeanalyse med MR og PyTorch

I denne notebook-en skal vi bygge en dyplæringsmodell for å analysere medisinske bilder. Vi går bort fra røntgenbildene vi så på tidligere, og fokuserer nå på **magnetisk resonanstomografi (MR)**-bilder av hjernen.

**Mål:** Vi skal trene et **Convolutional Neural Network (CNN)** til å klassifisere MR-bilder og skille mellom hjerner fra friske kontrollpersoner og personer med demens.

**Datasett:** Vi bruker et lite utvalg fra [OASIS-1](https://www.oasis-brains.org/)-datasettet. Dataene består av 3D MR-bilder i NIfTI-format (`.nii`). For å gjøre oppgaven enklere, vil vi trekke ut ett 2D-snitt fra midten av hver 3D-skanning for å utføre 2D-bildeklassifisering.

**Verktøy:**
- **PyTorch:** Et populært rammeverk for dyp læring.
- **Nibabel:** Et Python-bibliotek for å lese og skrive vanlige medisinske bildeformater, som NIfTI.
- **Scikit-learn:** For datasplitting og evaluering.
- **Matplotlib:** For visualisering.

### Steg 1: Importere Nødvendige Biblioteker

Først importerer vi alle bibliotekene vi trenger for databehandling, modellbygging, trening og evaluering.

In [7]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Sjekk om GPU eller MPS er tilgjengelig og sett enhet (device)
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print(f'Bruker enhet: {device}')

Bruker enhet: mps


In [8]:
print(f"PyTorch version: {torch.__version__}")
print(f"MPS tilgjengelig: {torch.backends.mps.is_available()}")
print(f"MPS bygget: {torch.backends.mps.is_built()}")

PyTorch version: 2.6.0
MPS tilgjengelig: True
MPS bygget: True


### Steg 2: Laste inn og Utforske Dataene

Vi antar at dataene er organisert i en mappestruktur som dette:
```
data/
└── oasis_mri_sample/
    ├── demented/
    │   └── subject_1.nii
    └── nondemented/
        └── subject_2.nii
```

Vi lager en funksjon som leser alle `.nii`-filene, trekker ut det midterste 2D-snittet fra 3D-volumet, og lagrer bildene sammen med sine etiketter (0 for `nondemented`, 1 for `demented`).

### En robust og standardisert metode: å laste ned dataene direkte fra Kaggle ved hjelp av deres API.
Dette er standard, garantert å fungere, og en veldig nyttig ferdighet å kunne. Det krever et lite, engangs-oppsett fra din side, men da vil du ha en stabil løsning.

#### Steg 1: Skaff din Kaggle API-nøkkel (tar 30 sekunder)

- Logg inn på din Kaggle-konto (eller opprett en gratis).
- Gå til din kontoside ved å klikke på profilikonet ditt og velge "Account".
- Scroll ned til seksjonen som heter "API".
- Klikk på knappen "Create New API Token".
- Dette vil laste ned en fil som heter kaggle.json. Ta vare på denne filen.

#### Steg 2: Oppdatert Python-skript

Nå skal vi bruke kaggle.json-filen i koden. Når du kjører skriptet under, må kaggle.json ligge i samme mappe, eller du må laste den opp til din Jupyter/Colab-økt.

Dette skriptet vil:
- Installere Kaggle-biblioteket.
- Sette opp API-tilgangen ved hjelp av din kaggle.json-fil.
- Laste ned det korrekte datasettet.
- Reorganisere filene slik din notebook forventer dem.

Hvordan du bruker den nye koden:

- Få kaggle.json som beskrevet i Steg 1.
- Plasser kaggle.json i samme mappe som du kjører Python-skriptet fra.
- Kjør skriptet. Det vil nå autentisere seg mot Kaggle og laste ned dataene på en stabil og sikker måte.

Denne metoden er standarden i feltet og vil garantert fungere så lenge Kaggle-datasettet er tilgjengelig. 

In [ ]:
import os
import subprocess
import json
import shutil
from tqdm import tqdm

def setup_kaggle_api():
    """
    Sets up the Kaggle API credentials from kaggle.json.
    """
    if not os.path.exists('kaggle.json'):
        print("Feil: `kaggle.json` ble ikke funnet.")
        print("Vennligst følg steg 1 i instruksjonene for å laste ned filen,")
        print("og plasser den i samme mappe som dette skriptet.")
        return False
        
    print("Setter opp Kaggle API-nøkkel...")
    # Create the .kaggle directory
    kaggle_dir = os.path.expanduser('~/.kaggle')
    os.makedirs(kaggle_dir, exist_ok=True)
    
    # Move the file and set permissions
    shutil.copy('kaggle.json', os.path.join(kaggle_dir, 'kaggle.json'))
    os.chmod(os.path.join(kaggle_dir, 'kaggle.json'), 0o600)
    
    # Import kaggle library after setup
    try:
        global kaggle
        import kaggle
        return True
    except ImportError:
        print("Kaggle-biblioteket er ikke installert. Prøver å installere...")
        subprocess.run(["pip", "install", "-q", "kaggle"])
        import kaggle
        return True
    except Exception as e:
        print(f"En feil oppstod under import av kaggle: {e}")
        return Fals

In [ ]:
def download_and_setup_mri_data():
    """
    Downloads, extracts, and reorganizes MRI data from Kaggle for
    binary classification (demented vs. nondemented).
    """
    if not setup_kaggle_api():
        return

    # --- Configuration ---
    base_dir = './data/oasis_mri_sample'
    temp_extract_folder = os.path.join(base_dir, 'temp_extracted')
    dataset_id = 'tourist55/alzheimers-dataset-4-class-of-images'
    zip_file_name = 'alzheimers-dataset-4-class-of-images.zip'

    # Create base directory
    os.makedirs(base_dir, exist_ok=True)

    print(f"\nLaster ned datasettet '{dataset_id}' fra Kaggle...")
    
    try:
        # Download the dataset
        kaggle.api.dataset_download_files(dataset_id, path=base_dir, unzip=False, quiet=False)

        # Unzip the file
        zip_path = os.path.join(base_dir, zip_file_name)
        print(f"\nPakker ut '{zip_file_name}'...")
        with kaggle.api.zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_extract_folder)

        print("Reorganiserer filene til 'demented' og 'nondemented'...")
        
        # --- Reorganization Logic ---
        demented_dest = os.path.join(base_dir, 'demented')
        nondemented_dest = os.path.join(base_dir, 'nondemented')
        os.makedirs(demented_dest, exist_ok=True)
        os.makedirs(nondemented_dest, exist_ok=True)
        
        # The source has an extra 'Alzheimer_s Dataset' folder
        source_root = os.path.join(temp_extract_folder, 'Alzheimer_s Dataset')
        
        # Copy 'NonDemented' files from the 'train' folder
        nondemented_src = os.path.join(source_root, 'train', 'NonDemented')
        for filename in tqdm(os.listdir(nondemented_src), desc="Kopierer NonDemented"):
            shutil.copy(os.path.join(nondemented_src, filename), nondemented_dest)
            
        # Combine all 'Demented' categories from the 'train' folder
        demented_categories = ['VeryMildDemented', 'MildDemented', 'ModerateDemented']
        for category in demented_categories:
            category_src = os.path.join(source_root, 'train', category)
            for filename in tqdm(os.listdir(category_src), desc=f"Kopierer {category}"):
                shutil.copy(os.path.join(category_src, filename), demented_dest)

    except Exception as e:
        print(f"En feil oppstod: {e}")
        print("Sjekk at du har skrevet inn riktig datasett-ID og at `kaggle.json` er gyldig.")
    finally:
        # --- Cleanup ---
        print("\nRydder opp midlertidige filer...")
        zip_path = os.path.join(base_dir, zip_file_name)
        if os.path.exists(zip_path):
            os.remove(zip_path)
        if os.path.exists(temp_extract_folder):
            shutil.rmtree(temp_extract_folder)

    # --- Verification ---
    try:
        demented_count = len(os.listdir(demented_dest))
        nondemented_count = len(os.listdir(nondemented_dest))
        
        print(f"\nOppsett fullført! Data er organisert:")
        print(f"- {demented_count} bilder i mappen 'demented'")
        print(f"- {nondemented_count} bilder i mappen 'nondemented'")
    except NameError:
         print("\nReorganisering ble ikke fullført på grunn av en tidligere feil.")
    except FileNotFoundError:
        print("\nKunne ikke telle filer. Noe gikk galt under reorganiseringen.")



In [ ]:
# Eksempel på hvordan funksjonen kjøres

# Vi sjekker om data allrede eksiterer for å unngå ny nedlastning
data_dir_check = './data/oasis_mri_sample/demented'
if not os.path.exists(data_dir_check) or not os.listdir(data_dir_check):
    download_and_setup_mri_data()
else:
    print("Data ser ut til å allerede eksistere. Hopper over nedlasting.")


Laster ned MRI-data fra ny kilde...
Feil under nedlasting: 404 Client Error: Not Found for url: https://github.com/Mina-Karam/Alzheimer-s-MRI-Predection/raw/main/Dataset.zip
Rydder opp midlertidige filer...


In [11]:
def load_mri_data(data_path):
    """Laster inn MR-data og returnerer 2D-snitt og etiketter."""
    images = []
    labels = []
    
    for label, category in enumerate(['nondemented', 'demented']):
        category_path = os.path.join(data_path, category)
        if not os.path.isdir(category_path):
            print(f"Advarsel: Mappen {category_path} ble ikke funnet.")
            continue
            
        for filename in os.listdir(category_path):
            if filename.endswith('.nii'):
                img_path = os.path.join(category_path, filename)
                try:
                    # Last inn NIfTI-filen
                    nii_img = nib.load(img_path)
                    img_data = nii_img.get_fdata()
                    
                    # Hent ut det midterste snittet langs z-aksen
                    mid_slice_idx = img_data.shape[2] // 2
                    mid_slice = img_data[:, :, mid_slice_idx]
                    
                    # Roter bildet for riktig visning
                    mid_slice = np.rot90(mid_slice)
                    
                    images.append(mid_slice)
                    labels.append(label)
                except Exception as e:
                    print(f"Kunne ikke laste {img_path}: {e}")
    
    return np.array(images), np.array(labels)

# Angi stien til dataene (endre denne om nødvendig)
DATA_DIR = '../../data/oasis_mri_sample/' # Antar at dataene ligger i en mappe på samme nivå som `uke03`

# Opprett datamapper hvis de ikke eksisterer
os.makedirs(os.path.join(DATA_DIR, 'nondemented'), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, 'demented'), exist_ok=True)

images, labels = load_mri_data(DATA_DIR)

if len(images) > 0:
    print(f'Lastet inn {len(images)} bilder.')
    print(f'Bildedimensjoner: {images[0].shape}')
    print(f'Antall i hver klasse: {np.bincount(labels)}')
else:
    print('Ingen bilder ble lastet inn. Sjekk stien til dataene og mappestrukturen.')

Ingen bilder ble lastet inn. Sjekk stien til dataene og mappestrukturen.


#### Visualisere Eksempelbilder

Det er alltid lurt å se på dataene for å få en intuisjon for hva modellen skal lære. Vi viser ett eksempel fra hver klasse.

In [ ]:
if len(images) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
    # Finn første bilde av en frisk person
    nondemented_idx = np.where(labels == 0)[0][0]
    axes[0].imshow(images[nondemented_idx], cmap='gray')
    axes[0].set_title('Klasse: Nondemented (Frisk)')
    axes[0].axis('off')
    
    # Finn første bilde av en dement person
    demented_idx = np.where(labels == 1)[0][0]
    axes[1].imshow(images[demented_idx], cmap='gray')
    axes[1].set_title('Klasse: Demented (Demens)')
    axes[1].axis('off')
    
    plt.show()

### Steg 3: Forprosessering og Klargjøring av Data

For at et nevralt nettverk skal kunne behandle bildene, må vi:
1.  **Dele dataene** inn i et treningssett og et valideringssett.
2.  **Opprette en egendefinert `Dataset`-klasse** i PyTorch. Dette er standard praksis for å håndtere data effektivt.
3.  **Definere transformasjoner:** Bildene må konverteres til PyTorch-tensorer, få endret størrelse til en fast dimensjon (f.eks. 128x128), og normaliseres.

In [ ]:
# 1. Del dataene i trenings- og valideringssett
X_train, X_val, y_train, y_val = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f'Størrelse på treningssett: {len(X_train)}')
print(f'Størrelse på valideringssett: {len(X_val)}')

# 2. Lag en egendefinert Dataset-klasse
class MRIDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        # Legg til en kanal-dimensjon (for gråtonebilder)
        image = np.expand_dims(image, axis=-1)
        
        if self.transform:
            image = self.transform(image)
            
        return image, torch.tensor(label, dtype=torch.long)

# 3. Definer transformasjoner
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128)),
    # Normalisering med gjennomsnitt og standardavvik for datasettet
    # For enkelhets skyld bruker vi (0.5, 0.5) som er vanlig for bilder i [-1, 1] området
    transforms.Normalize((0.5,), (0.5,))
])

# Opprett Dataset- og DataLoader-objekter
train_dataset = MRIDataset(X_train, y_train, transform=data_transforms)
val_dataset = MRIDataset(X_val, y_val, transform=data_transforms)

BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

### Steg 4: Bygge CNN-Modellen

Nå definerer vi arkitekturen til vårt nevrale nettverk. Vi lager en enkel CNN-modell med to konvolusjonslag etterfulgt av to fullt-tilkoblede (dense) lag.

**Arkitektur:**
1.  `Conv2d` -> `ReLU` -> `MaxPool2d` (Første konvolusjonsblokk)
2.  `Conv2d` -> `ReLU` -> `MaxPool2d` (Andre konvolusjonsblokk)
3.  `Flatten` (Gjør om 2D-kart til 1D-vektor)
4.  `Linear` -> `ReLU` (Første tette lag)
5.  `Linear` (Output-lag, som gir en score for hver klasse)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        # Etter to pooling-lag vil et 128x128 bilde bli 32x32
        # Størrelsen på input til det lineære laget er 32 * 32 * 32 (kanaler)
        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, 2) # 2 klasser: nondemented og demented
        )

    def forward(self, x):
        x = self.conv_layer1(x)
        x = self.conv_layer2(x)
        x = self.fc_layer(x)
        return x

# Initialiser modellen og flytt den til GPU hvis tilgjengelig
model = SimpleCNN().to(device)
print(model)

### Steg 5: Trene Modellen

Nå er vi klare til å trene modellen. Vi må definere:
- **En tapsfunksjon (Loss Function):** Måler hvor feil modellen predikerer. `CrossEntropyLoss` er standard for klassifiseringsoppgaver.
- **En optimaliseringsalgoritme (Optimizer):** Oppdaterer vektene i nettverket for å minimere tapet. `Adam` er et robust og populært valg.

Deretter skriver vi en treningsløkke som itererer over dataene i et visst antall *epochs*. For hver epoch trener vi på treningssettet og evaluerer på valideringssettet for å overvåke ytelsen.

In [ ]:
# Definer hyperparametere
LEARNING_RATE = 0.001
EPOCHS = 20

# Definer tapsfunksjon og optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

for epoch in range(EPOCHS):
    # --- Treningsfase ---
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Nullstill gradienter
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass og optimalisering
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        
    train_loss = running_loss / len(train_loader.dataset)
    train_acc = correct_train / total_train
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)

    # --- Valideringsfase ---
    model.eval()
    running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad(): # Ingen grunn til å beregne gradienter her
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = running_loss / len(val_loader.dataset)
    val_acc = correct_val / total_val
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    print(f'Epoch {epoch+1}/{EPOCHS} | ' \
          f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | ' \
          f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')

### Steg 6: Evaluere Modellen

Etter treningen er det viktig å evaluere modellens ytelse. Vi gjør dette på flere måter:
1.  **Plotte treningshistorikk:** Vi ser på hvordan tap og nøyaktighet utvikler seg over tid for både trenings- og valideringssettet. Dette kan avsløre problemer som *overfitting*.
2.  **Forvirringsmatrise (Confusion Matrix):** Viser en tabell over hvor mange bilder som ble korrekt og feilaktig klassifisert for hver klasse. Dette gir et mer detaljert bilde enn bare nøyaktighet.
3.  **Klassifiseringsrapport:** Gir en oppsummering av presisjon, sensitivitet (recall) og F1-score for hver klasse.

In [ ]:
# 1. Plotte treningshistorikk
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(history['train_loss'], label='Training Loss')
ax1.plot(history['val_loss'], label='Validation Loss')
ax1.set_title('Loss over Epochs')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()

ax2.plot(history['train_acc'], label='Training Accuracy')
ax2.plot(history['val_acc'], label='Validation Accuracy')
ax2.set_title('Accuracy over Epochs')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()

plt.show()

In [ ]:
# 2 & 3. Forvirringsmatrise og klassifiseringsrapport
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

class_names = ['Nondemented', 'Demented']
print("Klassifiseringsrapport:")
print(classification_report(all_labels, all_preds, target_names=class_names))

cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.title('Forvirringsmatrise')
plt.show()

### Konklusjon

I denne notebook-en har vi bygget en ende-til-ende-løsning for klassifisering av MR-bilder ved hjelp av PyTorch.

Vi har:
1.  Lastet inn og forprosessert medisinske bilder i NIfTI-format.
2.  Definert en egendefinert `Dataset`-klasse for å håndtere dataene i PyTorch.
3.  Bygget og trent en enkel CNN-modell fra bunnen av.
4.  Evaluert modellens ytelse med relevante metrikker som nøyaktighet, forvirringsmatrise og klassifiseringsrapport.

Dette er et startpunkt. For å forbedre modellen kan man utforske:
- **Dataaugmentering:** Roter, zoom eller flipp bildene for å kunstig øke størrelsen på treningssettet.
- **Mer avanserte arkitekturer:** Bruk forhåndstrente modeller som ResNet (transfer learning).
- **3D-konvolusjoner:** Utnytt den fulle 3D-informasjonen i MR-skanningene ved å bruke `Conv3d`-lag.